In [1]:
import warnings
import sys
import pathlib
import torch
import gpytorch
from IPython.display import HTML


warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

import loaders.hrrr
import fields.continuous

In [2]:
import loaders.hrrr
import fields.discrete
import fields.continuous
import models.gp
import models.neural_flow

In [3]:
hrrr = loaders.hrrr.Data(date="2024-09-26 17:00", hours=4, level= 700)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws


In [4]:
dvf = hrrr.vector_field(extent=(-88,-81,24.5,29))
dsf = hrrr.scalar_field(extent=(-88,-81,24.5,29))

In [5]:
dvf.coarsen(4)
dsf.coarsen(4)

In [6]:
HTML(dsf.plot().to_html5_video())

In [7]:
HTML(dvf.plot().to_html5_video())

In [77]:
cvf0 = fields.continuous.VectorField()
cvf0.train_mse(dvf, batch_size=2000, epochs = 50)

Iter 1/400 — loss: 0.2702
Iter 2/400 — loss: 0.2618
Iter 3/400 — loss: 0.2580
Iter 4/400 — loss: 0.2581
Iter 5/400 — loss: 0.2549
Iter 6/400 — loss: 0.2505
Iter 7/400 — loss: 0.2482
Iter 8/400 — loss: 0.2490
Iter 9/400 — loss: 0.2469
Iter 10/400 — loss: 0.2429
Iter 11/400 — loss: 0.2400
Iter 12/400 — loss: 0.2372
Iter 13/400 — loss: 0.2366
Iter 14/400 — loss: 0.2304
Iter 15/400 — loss: 0.2262
Iter 16/400 — loss: 0.2259
Iter 17/400 — loss: 0.2212
Iter 18/400 — loss: 0.2099
Iter 19/400 — loss: 0.2091
Iter 20/400 — loss: 0.2023
Iter 21/400 — loss: 0.1869
Iter 22/400 — loss: 0.1821
Iter 23/400 — loss: 0.1784
Iter 24/400 — loss: 0.1662
Iter 25/400 — loss: 0.1526
Iter 26/400 — loss: 0.1457
Iter 27/400 — loss: 0.1408
Iter 28/400 — loss: 0.1357
Iter 29/400 — loss: 0.1308
Iter 30/400 — loss: 0.1307
Iter 31/400 — loss: 0.1295
Iter 32/400 — loss: 0.1276
Iter 33/400 — loss: 0.1250
Iter 34/400 — loss: 0.1143
Iter 35/400 — loss: 0.1109
Iter 36/400 — loss: 0.1106
Iter 37/400 — loss: 0.1094
Iter 38/40

In [78]:
HTML(cvf0.plot(dvf.coord_field).to_html5_video())

In [121]:
dsf0 = fields.discrete.ScalarField(dvf.coord_field)
flow = models.neural_flow.ODE(cvf0.velocity)
dsf0.simulate(flow)

In [122]:
HTML(dsf0.plot().to_html5_video())

In [120]:
cvf = fields.continuous.VectorField()
cvf.train_mle(dsf0, batch_size=600, epochs = 10)

Iter 1/250 — Likelihood: 1.1532 — noise_var: 0.1010
Iter 2/250 — Likelihood: 1.3891 — noise_var: 0.1019
Iter 3/250 — Likelihood: 1.3157 — noise_var: 0.1028
Iter 4/250 — Likelihood: 1.2778 — noise_var: 0.1037
Iter 5/250 — Likelihood: 1.2532 — noise_var: 0.1047
Iter 6/250 — Likelihood: 1.2440 — noise_var: 0.1057
Iter 7/250 — Likelihood: 1.3516 — noise_var: 0.1066
Iter 8/250 — Likelihood: 1.2514 — noise_var: 0.1076
Iter 9/250 — Likelihood: 1.2485 — noise_var: 0.1086
Iter 10/250 — Likelihood: 1.3231 — noise_var: 0.1096
Iter 11/250 — Likelihood: 1.2310 — noise_var: 0.1106
Iter 12/250 — Likelihood: 1.2287 — noise_var: 0.1116
Iter 13/250 — Likelihood: 1.3224 — noise_var: 0.1126
Iter 14/250 — Likelihood: 1.2883 — noise_var: 0.1136
Iter 15/250 — Likelihood: 1.1675 — noise_var: 0.1145
Iter 16/250 — Likelihood: 1.2784 — noise_var: 0.1155
Iter 17/250 — Likelihood: 1.3014 — noise_var: 0.1165
Iter 18/250 — Likelihood: 1.1628 — noise_var: 0.1175
Iter 19/250 — Likelihood: 1.2229 — noise_var: 0.1184
It

In [117]:
HTML(cvf.plot(dvf.coord_field).to_html5_video())

In [115]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [113]:
cvf = fields.continuous.VectorField()
cvf.train_mle(dsf, batch_size=500, epochs = 7)

Iter 1/210 — Likelihood: 0.8393 — noise_var: 0.0991
Iter 2/210 — Likelihood: 0.7823 — noise_var: 0.0981
Iter 3/210 — Likelihood: 0.7731 — noise_var: 0.0972
Iter 4/210 — Likelihood: 0.7349 — noise_var: 0.0963
Iter 5/210 — Likelihood: 0.8104 — noise_var: 0.0954
Iter 6/210 — Likelihood: 0.7833 — noise_var: 0.0945
Iter 7/210 — Likelihood: 0.8006 — noise_var: 0.0937
Iter 8/210 — Likelihood: 0.7383 — noise_var: 0.0929
Iter 9/210 — Likelihood: 0.7429 — noise_var: 0.0921
Iter 10/210 — Likelihood: 0.7835 — noise_var: 0.0914
Iter 11/210 — Likelihood: 0.6986 — noise_var: 0.0906
Iter 12/210 — Likelihood: 0.7012 — noise_var: 0.0898


KeyboardInterrupt: 

In [27]:
HTML(cvf.plot(dvf.coord_field).to_html5_video())